In [13]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from collections import OrderedDict
import pandas as pd
# Make the graphs a bit prettier, and bigger
pd.set_option('display.mpl_style', 'default')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 20)

import numpy as np
import math
# The usual preamble
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt
plt.rcParams['axes.color_cycle'] = ['r', 'g', 'b', 'c']
plt.rcParams['lines.color'] = 'r'
plt.rcParams['figure.figsize'] = (15, 5)

import process
from modeling import categorize
import json

Populating the interactive namespace from numpy and matplotlib


In [14]:
import pickle
#clf = pickle.load(open('randomforestclassifier.p','rb'))
best_feature_names = pickle.load(open('best_features_names_reg_diversity_80_20_Train_Test_TrimData_1kTrees_5MinLeaves_MaxFeaturesAll.p','rb'))
model = pickle.load(open('randomforestclassifier_fixed_reg_diversity_80_20_Train_Test_TrimData_1kTrees_5MinLeaves_MaxFeaturesAll.p','rb'))
print model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=183, max_leaf_nodes=None, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=4, oob_score=True, random_state=None,
           verbose=0, warm_start=False)


In [15]:
best_feature_names

['totacad',
 'sgtmin',
 'diversity_index',
 'entrymin',
 'white_officer_disparity',
 'opbudget_per_capita',
 'sgtmax',
 'entrymax',
 'black_over_white_institutionalized_adult_disparity',
 'institutionalized_juvenile_white',
 'chiefmin',
 'numcarcam',
 'numfixcam_per_capita',
 'total_income_estimate_all_per_capita',
 'rural',
 'white_per_totofficers',
 'institutionalized_juvenile_all',
 'chiefmax',
 'numrespoff_per_capita',
 'black_over_white_institutionalized_disparity',
 'numcarcam_per_capita',
 'black_over_white_institutionalized_adult_local_jail_disparity',
 'numfixcam',
 'numsro_per_capita',
 'institutionalized_juvenile_white_per_capita',
 'institutionalized_juvenile_all_per_capita',
 'numothunm',
 'swnauthemp_per_capita',
 'civftemp',
 'civftemp_per_capita',
 'nummrkcars_per_capita',
 'swnauthemp',
 'hispanic_per_totofficers',
 'total_income_estimate_black_per_capita',
 'totinsrv',
 'drugforf_per_capita',
 'black_over_white_institutionalized_juvenile_disparity',
 'black_over_white

In [16]:
df_with_rpsi = process.add_features(process.get_data(munge=True,with_traffic=True))
#df_all = process.add_features(process.get_data(munge=False,with_traffic=False))
df_all = process.add_features(process.get_data(munge=True,with_traffic=False,drop_features=False))
df_all_drop = process.add_features(process.get_data(munge=True,with_traffic=False))

#df_all = df_all_munge.merge(df_all,how='left') #in case dropped some during munging
df_all = df_all.merge(pd.DataFrame(df_with_rpsi['rpsi']),how='left',right_index=True,left_index=True)
df_all['is_measured'] = np.isnan(df_all['rpsi'])==False
features = df_all_drop.columns.tolist()

In [17]:
_,val_df = process.get_split_add_data(frac=.4)
processor = process.Processor(val_df,categorize=False)
X_predict,_ = processor.get_scaled_Xy(df_all_drop) #[best_feature_names])
y_predict = model.predict(X_predict)
df_all_predictions = pd.DataFrame(y_predict,columns=['rpsi_predicted'],index=df_all_drop.index)
df_all_predictions['rpsi_predicted_value'] = df_all_predictions['rpsi_predicted']
df_all_predictions['rpsi_predicted'] = df_all_predictions['rpsi_predicted'].map(categorize)
df_all = df_all.merge(df_all_predictions,how='left',right_index=True,left_index=True)
df_all = df_all.fillna(-1)
df_all = df_all[((df_all['rpsi']==-1) & (df_all['rpsi_predicted']==-1))==False]
df_all


,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,numboats_per_capita,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,diversity_index,white_per_totofficers,black_per_totofficers,hispanic_per_totofficers,asian_per_totofficers,nathaw_per_totofficers,amerind_per_totofficers,multrace_per_totofficers,unkrace_per_totofficers,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted,rpsi_predicted_value
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Anchorage Police Department,Anchorage,AK,99507,3,399,385,0,174,0,559,0,0,0,0,0,6,0,12,0,6,0,7,0,182,6,18,182,76,0,...,0.000000,0.000050,0.000062,0.000000,0.000000,0.286080,0.841558,0.054545,0.036364,0.044156,0.01039,0.005195,0.007792,0.000000,0.092703,0.087302,0.131329,0.228111,1.000000,1.000000,1.000000,0.252708,0.148649,0.940382,1.345001,0.699168,-1,False,2,2.440543
1001,Ketchikan Police Department,Ketchikan,AK,99901,3,23,23,0,10,1,33,1,0,0,0,0,0,0,0,0,0,0,0,0,23,0,2,0,0,0,...,0.000000,0.000000,0.001036,0.000666,0.000000,0.387352,0.782609,0.000000,0.086957,0.086957,0.00000,0.043478,0.000000,0.000000,0.008583,0.005714,0.070175,0.000000,1.000000,1.000000,0.000000,1.000000,0.214286,0.000000,1.148574,0.000000,-1,False,2,2.421393
16,Tuscaloosa Police Department,Tuscaloosa,AL,35401,3,269,264,0,75,0,339,0,0,0,0,0,0,0,11,0,1,0,0,0,157,7,10,157,58,0,...,0.000036,0.000071,0.000337,0.000426,0.000009,0.282905,0.834586,0.146617,0.007519,0.003759,0.00000,0.000000,0.000000,0.007519,0.722283,0.688996,1.017279,1.487805,1.000000,1.000000,1.000000,1.487805,0.783784,0.369151,1.517748,0.243223,-1,False,2,2.753272
13,Mobile Police Department,Mobile,AL,36606,3,532,526,0,254,96,780,96,0,6,0,0,0,0,5,0,1,0,0,0,304,30,5,345,97,0,...,0.000004,0.000059,0.000369,0.000148,0.000000,0.427804,0.703422,0.279468,0.007605,0.005703,0.00000,0.003802,0.000000,0.000000,0.887893,0.763822,0.922970,1.873085,1.000000,1.000000,1.000000,1.873085,2.195122,0.622467,1.391109,0.447461,-1,False,2,2.441861
1004,Anniston Police Department,Anniston,AL,36202,3,92,81,0,33,0,114,0,0,0,0,0,0,0,1,0,0,0,0,0,69,0,0,0,0,0,...,0.000000,0.000014,0.000565,0.000085,0.000000,0.140432,0.925926,0.061728,0.000000,0.012346,0.00000,0.000000,0.000000,0.000000,0.392830,0.413985,0.371879,0.723881,1.000000,1.000000,1.000000,0.723881,0.551020,0.230880,1.360448,0.169709,-1,False,2,2.320553
5,Birmingham Police Department,Birmingham,AL,35203,3,914,768,0,400,0,1168,0,0,0,0,0,4,0,0,0,1,0,0,0,462,462,12,528,137,1,...,0.000000,0.000047,0.000132,0.000000,0.000000,0.474844,0.369792,0.623698,0.005208,0.001302,0.00000,0.000000,0.000000,0.000000,0.872831,0.813041,0.856183,1.704591,1.000000,1.000000,0.618497,2.277439,2.373333,1.421503,0.735633,1.932355,-1,False,1,2.116773
15,Montgomery Police Department,Montgomery,AL,36101,3,510,491,0,97,0,588,0,0,0,0,0,2,0,1,0,2,0,0,0,212,12,15,212,88,2,...,0.000000,0.000070,0.000415,0.000000,0.000061,0.505075,0.574338,0.407332,0.010183,0.008147,0.00000,0.000000,0.000000,0.000000,1.488363,1.322901,1.429854,1.529372,5.372549,1.235294,1.173724,4.019802,1.941748,0.723585,1.518510,0.476510,-1,False,2,2.715275
7,Dothan Police Department,Dothan,AL,36303,3,157,148,0,100,0,248,0,0,0,0,15,0,0,0,0,0,0,0,0,80,0,12,80,38,22,...,0

In [18]:
df_all['rpsi_final'] =  df_all['rpsi_predicted']
df_all['rpsi_final']
for index,series in df_all.iterrows():
    if series['is_measured']:
        df_all.loc[index,'rpsi_final'] = categorize(series['rpsi'])
df_all[df_all['is_measured']]

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,diversity_index,white_per_totofficers,black_per_totofficers,hispanic_per_totofficers,asian_per_totofficers,nathaw_per_totofficers,amerind_per_totofficers,multrace_per_totofficers,unkrace_per_totofficers,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted,rpsi_predicted_value,rpsi_final
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1212,New London Police Department,New London,CT,6320,3,86,78,0,0,0,78,0,0,0,0,0,0,0,2,0,0,0,0,0,51,2,2,0,0,0,...,0.000109,0.000435,0.000579,0.000000,0.355977,0.794872,0.102564,0.076923,0.012821,0.012821,0.000000,0.000000,0,0.288592,0.299111,0.126984,1.000000,1,1.000000,1.000000,1.000000,1.000000,0.588391,1.315987,0.447110,1.322864,True,1,1.898827,0
1222,Westport Police Department,Westport,CT,6880,3,65,65,15,13,36,78,51,0,15,0,36,0,0,0,0,1,0,0,0,37,0,0,0,0,0,...,0.000038,0.000075,0.000525,0.000000,0.089423,0.953846,0.000000,0.046154,0.000000,0.000000,0.000000,0.000000,0,0.013104,0.003862,0.256944,1.000000,1,1.000000,1.000000,1.000000,1.200000,0.000000,1.031163,0.000000,3.067020,True,2,2.389858,2
184,Waterbury Police Department,Waterbury,CT,6702,3,300,300,0,53,11,353,11,0,0,0,0,0,0,1,0,0,0,0,0,168,12,7,185,58,0,...,0.000082,0.000000,0.000027,0.000027,0.339331,0.800000,0.070000,0.130000,0.000000,0.000000,0.000000,0.000000,0,0.341569,0.290306,0.242741,0.987952,1,1.000000,1.000000,1.000000,0.565217,0.348868,1.361861,0.256170,1.356672,True,1,1.695435,0
175,Hartford Police Department,Hartford,CT,6120,3,592,417,0,92,97,509,97,0,0,0,0,2,0,3,0,1,0,0,0,150,64,8,250,65,15,...,0.000105,0.000347,0.000016,0.000000,0.545690,0.625899,0.151079,0.201439,0.021583,0.000000,0.000000,0.000000,0,1.314375,1.031085,0.665044,1.354497,1,1.000000,1.000000,1.411003,1.000000,0.389109,2.118801,0.183646,1.708121,True,2,2.213556,1
1223,Wethersfield Police Department,Wethersfield,CT,6109,3,47,47,0,11,23,58,23,0,0,0,0,0,0,1,0,0,0,0,0,35,3,1,0,0,0,...,0.000075,0.000000,0.000037,0.000000,0.268270,0.851064,0.042553,0.106383,0.000000,0.000000,0.000000,0.000000,0,0.035182,0.043460,0.081871,1.000000,1,1.000000,1.000000,1.000000,0.000000,1.350963,0.950585,1.421190,1.314888,True,1,2.118632,0
170,Danbury Police Department,Danbury,CT,6810,3,152,148,0,6,0,154,0,0,9,0,0,0,0,5,0,0,1,0,0,90,0,5,90,22,0,...,0.000000,0.000000,0.000000,0.000000,0.199577,0.891892,0.027027,0.074324,0.006757,0.000000,0.000000,0.000000,0,0.105144,0.083884,0.373494,0.637363,1,0.637363,1.000000,1.000000,0.411765,0.376702,1.307061,0.288205,1.210668,True,1,1.632167,0
174,Hamden Police Department,Hamden,CT,6518,3,107,101,0,25,25,126,25,0,1,0,0,1,0,1,0,0,0,0,0,60,3,2,62,28,0,...,0.000066,0.000197,0.000000,0.000016,0.133465,0.930693,0.029703,0.019802,0.019802,0.000000,0.000000,0.000000,0,0.294419,0.269127,0.146006,1.000000,1,1.000000,1.000000,1.000000,0.651163,0.147321,1.359057,0.108400,2.385800,True,1,1.997691,2
179,New Britain Police Department,New Britain,CT,6051,3,160,172,0,12,6,184,6,0,0,0,0,0,0,5,0,0,0,0,0,100,7,2,100,27,6,...,0.000055,0.000000,0.000000,0.000000,0.203794,0.889535,0.058140,0.052326,0.000000,0.000000,0.000000,0.000000,0,0.204476,0.140

In [6]:
df_all.to_sql('app_db',process.engine,if_exists='replace')

In [7]:
df_all.to_csv('app_db.csv')

In [8]:
df_all[df_all['agency'].str.contains('raleigh'.title())]

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,diversity_index,white_per_totofficers,black_per_totofficers,hispanic_per_totofficers,asian_per_totofficers,nathaw_per_totofficers,amerind_per_totofficers,multrace_per_totofficers,unkrace_per_totofficers,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted,rpsi_predicted_value,rpsi_final
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
594,Raleigh Police Dept,Raleigh,NC,27602,3,764,710,0,114,0,824,0,0,0,0,0,1,0,1,0,2,0,0,0,305,6,10,305,110,0,...,0.000043,0.000143,0,0,0.320944,0.811268,0.142254,0.029577,0.015493,0,0.001408,0,0,0.422356,0.378851,0.985024,1.365526,1,1,1.14524,1.757396,3.761905,0.547183,1.317992,0.415164,1.726268,True,1,1.754993,1


## split by quantiles for plotting

In [19]:
featuremap = OrderedDict()
featuremap['totacad'] = ['Required Officer Training','Hours']
featuremap['sgtmin'] = ['Minimum Sergeant  Salary','USD']
featuremap['diversity_index'] = ['Officer Diversity Index','Larger values mean more diversity']
featuremap['entrymin'] = ['Minimum Officer Salary','USD']

featuremap['white_officer_disparity'] = ['Officer Demographic Disparity','Fraction of white officers compared to population']
featuremap['opbudget_per_capita'] = ['Police Budget','USD per capita']

featuremap['sgtmax'] = ['Maximum Sergeant Salary','USD']
featuremap['entrymax'] = ['Maximum Officer Salary','USD']
#featuremap['numcarcam'] = ['Number of Patrol Car Cameras']
#featuremap['urban_per_capita'] = ['Urban residents','per capita']
#featuremap['population_black_per_capita'] = ['Black population','per capita']
#featuremap['chiefmin'] = ['Minimum Chief Officer Salary','USD']
#featuremap['total_income_estimate_all_per_capita'] = ['Total Income','USD per capita']
#featuremap['black_over_white_institutionalized_adult_disparity'] =  ['Black/White in Jail','']
#featuremap['total_income_estimate_white_per_capita'] = ['Total Income for whites','USD per capita']
#featuremap['nummrkcars_per_capita'] = ['Number of marked cars','per capita']
#featuremap['swnftemp_per_capita'] = ['Number of officers','per capita']
#featuremap['institutionalized_all_per_capita'] = ['Institutionalized residents','per capita']
#featuremap['black_over_white_population_disparity'] = ['Black/White population','']


In [11]:
df_all

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,diversity_index,white_per_totofficers,black_per_totofficers,hispanic_per_totofficers,asian_per_totofficers,nathaw_per_totofficers,amerind_per_totofficers,multrace_per_totofficers,unkrace_per_totofficers,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted,rpsi_predicted_value,rpsi_final
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Anchorage Police Department,Anchorage,AK,99507,3,399,385,0,174,0,559,0,0,0,0,0,6,0,12,0,6,0,7,0,182,6,18,182,76,0,...,0.000050,0.000062,0.000000,0.000000,0.286080,0.841558,0.054545,0.036364,0.044156,0.01039,0.005195,0.007792,0.000000,0.092703,0.087302,0.131329,0.228111,1.000000,1.000000,1.000000,0.252708,0.148649,0.940382,1.345001,0.699168,-1,False,2,2.440543,2
1001,Ketchikan Police Department,Ketchikan,AK,99901,3,23,23,0,10,1,33,1,0,0,0,0,0,0,0,0,0,0,0,0,23,0,2,0,0,0,...,0.000000,0.001036,0.000666,0.000000,0.387352,0.782609,0.000000,0.086957,0.086957,0.00000,0.043478,0.000000,0.000000,0.008583,0.005714,0.070175,0.000000,1.000000,1.000000,0.000000,1.000000,0.214286,0.000000,1.148574,0.000000,-1,False,2,2.421393,2
16,Tuscaloosa Police Department,Tuscaloosa,AL,35401,3,269,264,0,75,0,339,0,0,0,0,0,0,0,11,0,1,0,0,0,157,7,10,157,58,0,...,0.000071,0.000337,0.000426,0.000009,0.282905,0.834586,0.146617,0.007519,0.003759,0.00000,0.000000,0.000000,0.007519,0.722283,0.688996,1.017279,1.487805,1.000000,1.000000,1.000000,1.487805,0.783784,0.369151,1.517748,0.243223,-1,False,2,2.753272,2
13,Mobile Police Department,Mobile,AL,36606,3,532,526,0,254,96,780,96,0,6,0,0,0,0,5,0,1,0,0,0,304,30,5,345,97,0,...,0.000059,0.000369,0.000148,0.000000,0.427804,0.703422,0.279468,0.007605,0.005703,0.00000,0.003802,0.000000,0.000000,0.887893,0.763822,0.922970,1.873085,1.000000,1.000000,1.000000,1.873085,2.195122,0.622467,1.391109,0.447461,-1,False,2,2.441861,2
1004,Anniston Police Department,Anniston,AL,36202,3,92,81,0,33,0,114,0,0,0,0,0,0,0,1,0,0,0,0,0,69,0,0,0,0,0,...,0.000014,0.000565,0.000085,0.000000,0.140432,0.925926,0.061728,0.000000,0.012346,0.00000,0.000000,0.000000,0.000000,0.392830,0.413985,0.371879,0.723881,1.000000,1.000000,1.000000,0.723881,0.551020,0.230880,1.360448,0.169709,-1,False,2,2.320553,2
5,Birmingham Police Department,Birmingham,AL,35203,3,914,768,0,400,0,1168,0,0,0,0,0,4,0,0,0,1,0,0,0,462,462,12,528,137,1,...,0.000047,0.000132,0.000000,0.000000,0.474844,0.369792,0.623698,0.005208,0.001302,0.00000,0.000000,0.000000,0.000000,0.872831,0.813041,0.856183,1.704591,1.000000,1.000000,0.618497,2.277439,2.373333,1.421503,0.735633,1.932355,-1,False,1,2.116773,1
15,Montgomery Police Department,Montgomery,AL,36101,3,510,491,0,97,0,588,0,0,0,0,0,2,0,1,0,2,0,0,0,212,12,15,212,88,2,...,0.000070,0.000415,0.000000,0.000061,0.505075,0.574338,0.407332,0.010183,0.008147,0.00000,0.000000,0.000000,0.000000,1.488363,1.322901,1.429854,1.529372,5.372549,1.235294,1.173724,4.019802,1.941748,0.723585,1.518510,0.476510,-1,False,2,2.715275,2
7,Dothan Police Department,Dothan,AL,36303,3,157,148,0,100,0,248,0,0,0,0,15,0,0,0,0,0,0,0,0,80,0,12,80,38,22,...,0.000025,0.001243,0.000000,0.000000,0.175676,0.905405,0.074

In [20]:
splitby = [.20,.40,.60,0.80,.98] #0.95]
quantiles = []
quantiles.append(df_all.quantile(splitby))
quantiles.append(df_all[df_all['rpsi_final']==0].quantile(splitby))
quantiles.append(df_all[df_all['rpsi_final']==1].quantile(splitby))
quantiles.append(df_all[df_all['rpsi_final']==2].quantile(splitby))
quantiles_df = pd.concat(quantiles)
#.to_json('test.json')
quantiles_df = quantiles_df[featuremap.keys()]
quantiles_df

,totacad,sgtmin,diversity_index,entrymin,white_officer_disparity,opbudget_per_capita,sgtmax,entrymax
0.20,480.00,37400.000000,0.029412,29850.000000,1.030247,69.450576,47592.000000,37500.000
0.40,618.00,45843.000000,0.123188,34627.000000,1.078452,109.554399,59134.000000,46854.290
0.60,720.00,54000.000000,0.226027,38860.000000,1.150427,154.149820,65790.000000,52973.000
0.80,880.00,66000.000000,0.390981,44602.000000,1.297847,218.052977,77750.000000,62483.000
0.98,1120.00,97982.000000,0.605676,65346.800000,1.930865,549.123253,110500.800000,87000.000
0.20,618.00,31901.600000,0.059211,27973.600000,1.104673,55.314435,44224.600000,37238.800
0.40,640.00,35721.200000,0.131628,29929.600000,1.188098,87.784755,52614.400000,43780.800
0.60,655.40,40011.600000,0.205126,31995.800000,1.266739,109.456942,59604.133341,47497.132
0.80,795.80,51583.000000,0.287060,35980.293022,1.410455,146.510787,67383.376000,55862.000
0.98,1148.02,72228.460000,0.486668,53988.300000,1.904233,271.246970,89564.640000,71467.980


In [21]:
def build_bullet_json(quantiles_df, all_df,surveyid=348,category=0):
    json_output = []
    indices_all = [0,1,2,3,4]
    indices_cat = []
    for i in indices_all:
        indices_cat.append(i+(category+1)*len(indices_all))
    for index,series in quantiles_df.iteritems():
        #assumes 4 quantiles
        if index not in featuremap: continue
        title = featuremap[index][0]
        subtitle = featuremap[index][1]
        quantiles_all = []
        for i in indices_all:
            quantiles_all.append(series.iloc[i])
        quantiles_cat = []
        for i in indices_cat:
            quantiles_cat.append(series.iloc[i])
        json_output.append({})
        json_output[-1]['title'] = title
        json_output[-1]['subtitle'] = subtitle
        json_output[-1]['markers'] = all_df[all_df.index==surveyid][index].tolist()
        json_output[-1]['markers_tmp'] = [0] #all_df[all_df.index==surveyid][index].tolist()
        json_output[-1]['ranges'] = quantiles_all
        json_output[-1]['ranges_tmp'] =[0,0,0,0,0] #quantiles_all

        json_output[-1]['measures'] = quantiles_cat
        json_output[-1]['measures_tmp'] = [0,0,0,0,0] #quantiles_cat

        json_output[-1]['dummy'] = [0] # quantiles_cat
        

    return json_output

        

In [22]:
for index,series in df_all.iterrows():
    category = int(series['rpsi_final'])
    surveyid = index
    output = build_bullet_json(quantiles_df,df_all,
                               surveyid=surveyid,category=category)
    output = json.dumps(output).replace('\\"','"').\
        replace('"[','[').replace(']"',']')
    with open('json/'+str(surveyid)+'.json', 'w') as outfile:
        outfile.write(output)
    